In [ ]:
import pandas as pd

# Assuming df is your DataFrame after reading from the initial CSV
df_choice = pd.read_csv(r'C:\Users\student\Desktop\JOSAA\data\Round1-20240508T102918Z-001\Round1\Choice.csv')

# Slice the DataFrame to keep only the top 10,000 rows
df_top10k = df[:10000]

# Save the top 10,000 rows to a new CSV file
df_top10k.to_csv(r'C:\Users\student\Desktop\JOSAA\data\Round1-20240508T102918Z-001\Round1\Choice_trial.csv', index=False)


In [ ]:
df = df[:10000]
df.to_csv(r'C:\Users\student\Desktop\JOSAA\data\Round1-20240508T102918Z-001\Round1\Choice_trial.csv', sep=';')


In [ ]:
df_candidate = pd.read_csv(r'C:\Users\student\Desktop\JOSAA\data\Round1-20240508T102918Z-001\Round1\Candidate.csv')
df_Choice_trial = pd.read_csv(r'C:\Users\student\Desktop\JOSAA\data\Round1-20240508T102918Z-001\Round1\Choice_trial.csv')
df_institute = pd.read_csv(r'C:\Users\student\Desktop\JOSAA\data\Round1-20240508T102918Z-001\Round1\Institute.csv') 
df_seats = pd.read_csv(r'C:\Users\student\Desktop\JOSAA\data\Round1-20240508T102918Z-001\Round1\Seats.csv')

In [ ]:


print("df_Choice_trial:")
print(df_Choice_trial)


In [ ]:
print("df_candidate:")
print(df_candidate)



In [ ]:
print("df_institute:")
print(df_institute)



In [ ]:
print("df_seats:")
print(df_seats)

In [22]:
pip install pyodbc

Note: you may need to restart the kernel to use updated packages.


In [36]:
import pyodbc
from collections import defaultdict, deque

class Program:
    def __init__(self, inst_code, brcode, capacity):
        self.inst_code = inst_code
        self.brcode = brcode
        self.capacity = capacity
        self.waitlist = []  # Will store tuples (roll_no, rank)

# Example dictionaries for demonstration
class Candidate:
    def __init__(self, roll_no, adv, mains):
        self.roll_no = roll_no
        self.adv = adv
        self.mains = mains
        self.current_index = 1

# Define your database connection parameters
server = 'localhost\\MSSQLSERVER01'
database = 'Round1'
conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
cursor = conn.cursor()

# Function to fetch candidate data from the database
def fetch_candidates():
    candidates_dict = {}
    cursor.execute(
    '''
    SELECT TOP 20 RollNo, CAST(Adv_CRL_Rank AS INT), CAST(AI_Eng_CRL_Rank AS INT)
    FROM Candidate
    WHERE CAST(Adv_CRL_Rank AS INT) > 0 
        AND Gender = 'B' 
        AND Cat = 'OP' 
        AND SubCat = 'NO' 
    ORDER BY CAST(Adv_CRL_Rank AS INT) ASC
    '''
)
    rows = cursor.fetchall()
    for row in rows:
        roll_no = row[0]
        adv_rank = row[1]
        mains_rank = row[2]
        candidates_dict[roll_no] = Candidate(roll_no, adv_rank, mains_rank)
    return candidates_dict

# Fetch candidates from the database
candidates = fetch_candidates()

for roll_no in candidates:
    print(f"Roll Number: {roll_no}, Adv Rank: {candidates[roll_no].adv}, AI Rank: {candidates[roll_no].mains}")

def fetch_programs():
    program = {}
    cursor.execute(
    '''
    SELECT InstCode, Brcode, TotalSeat
    FROM Seats
    WHERE Gender = 'B' 
        AND Cat = 'OP' 
        AND SubCat = 'NO' 
    '''
)
    rows = cursor.fetchall()
    for row in rows:
        instCode = row[0]
        brCode = row[1]
        capacity = int(row[2])
        program[instCode, brCode] = Program(instCode, brCode, capacity)
    return program

programs = fetch_programs()

# for instCode, brCode in programs:
#     print(f'{instCode} - {brCode} - {programs[instCode, brCode].capacity}')


Roll Number: 230310481885, Adv Rank: 1, AI Rank: 15
Roll Number: 230310127660, Adv Rank: 2, AI Rank: 28
Roll Number: 230310141436, Adv Rank: 3, AI Rank: 19
Roll Number: 230310400307, Adv Rank: 4, AI Rank: 20
Roll Number: 230310109987, Adv Rank: 5, AI Rank: 95
Roll Number: 230310522471, Adv Rank: 6, AI Rank: 61
Roll Number: 230310146952, Adv Rank: 7, AI Rank: 16
Roll Number: 230310132455, Adv Rank: 8, AI Rank: 4
Roll Number: 230310107985, Adv Rank: 9, AI Rank: 65
Roll Number: 230310025905, Adv Rank: 11, AI Rank: 350
Roll Number: 230310190221, Adv Rank: 13, AI Rank: 109
Roll Number: 230310180674, Adv Rank: 14, AI Rank: 24
Roll Number: 230310208555, Adv Rank: 15, AI Rank: 337
Roll Number: 230310046630, Adv Rank: 16, AI Rank: 242
Roll Number: 230310129610, Adv Rank: 17, AI Rank: 27
Roll Number: 230310434666, Adv Rank: 18, AI Rank: 43
Roll Number: 230310112527, Adv Rank: 19, AI Rank: 75
Roll Number: 230310056654, Adv Rank: 20, AI Rank: 102
Roll Number: 230310025597, Adv Rank: 21, AI Rank: 5

In [37]:
# Create an empty queue of candidates
queue = deque([candidates[roll_no] for roll_no in candidates])


In [25]:
import pyodbc

server = 'localhost\\MSSQLSERVER01'
database = 'Round1'
conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
cursor = conn.cursor()

def getPreference(roll_no, choice):
    cursor.execute(
        '''
        SELECT InstCode, BrCode
        FROM Choice
        WHERE RollNo = ? 
            AND CAST(ChoiceNo AS INT) = ?
        ''',
        (roll_no, choice)  # Pass the actual values here
    )
    rows = cursor.fetchall()
    if rows:
        instCode, brCode = rows[0]
        return programs[instCode, brCode]
    else:
        return None  # or handle the case where no rows are returned

# Example usage
prgm = getPreference(230310481885, 1)
if prgm:
    print(f'{prgm.capacity}')
else:
    print("No Program")


59


In [38]:
def process_candidates(programs, candidates, queue):
    # Allocation dictionary to track which program each candidate ends up with
    allocations = {}

    while queue:
        candidate = queue.popleft()
        program = getPreference(candidate.roll_no, candidates[candidate.roll_no].current_index)

        # Ensure program exists and candidate is eligible (capacity check here for simplicity)
        if program and program.capacity > 0:
            # Simulate allocation
            program.waitlist.append(candidate.roll_no)
            allocations[candidate.roll_no] = program
            # program.capacity -= 1  # Decrement the available seats
            
            # Handle over-capacity by removing the last candidate (simplest method for demonstration)
            if len(program.waitlist) > program.capacity:
                removed_roll_no = program.waitlist.pop(0)  # Remove the first candidate to simulate FIFO
                allocations.pop(removed_roll_no, None)  # Remove from allocations
                candidates[removed_roll_no].current_index += 1  # Move to next preference
                queue.append(candidates[removed_roll_no])  # Re-enqueue the candidate

    return allocations, programs

# Assuming `programs` and `candidates` are already initialized and `queue` is set up:
allocations, final_programs = process_candidates(programs, candidates, queue)

# Printing the results
print("Final Allocations:")
for roll_no, program_key in allocations.items():
    print(f"Candidate {roll_no} is allocated to program {program_key.inst_code}\'s  {program_key.brcode}")

print("\nFinal Waitlists:")
for program_key, program in final_programs.items():
    print(f"Program {program_key} waitlist: {program.waitlist} with {program.capacity} remaining seats")


Final Allocations:
Candidate 230310481885 is allocated to program 102's  4110
Candidate 230310127660 is allocated to program 102's  4110
Candidate 230310141436 is allocated to program 102's  4110
Candidate 230310400307 is allocated to program 102's  4110
Candidate 230310109987 is allocated to program 102's  4110
Candidate 230310522471 is allocated to program 102's  4110
Candidate 230310146952 is allocated to program 102's  4110
Candidate 230310132455 is allocated to program 102's  4110
Candidate 230310107985 is allocated to program 102's  4110
Candidate 230310025905 is allocated to program 102's  4110
Candidate 230310190221 is allocated to program 102's  4110
Candidate 230310180674 is allocated to program 102's  4110
Candidate 230310208555 is allocated to program 102's  4110
Candidate 230310046630 is allocated to program 104's  4110
Candidate 230310129610 is allocated to program 102's  4110
Candidate 230310434666 is allocated to program 102's  4110
Candidate 230310112527 is allocated t